# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

# Librerías

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [30]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [31]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [32]:
print(data_dir)

C:\Users\christian.espinosa\.keras\datasets\20_newsgroup


In [33]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['37261', '37913', '37914', '37915', '37916']


In [34]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "37261").read())

Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:37261 alt.graphics:519 comp.graphics.animation:2614
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!zaphod.mps.ohio-state.edu!darwin.sura.net!dtix.dt.navy.mil!oasys!lipman
From: lipman@oasys.dt.navy.mil (Robert Lipman)
Newsgroups: comp.graphics,alt.graphics,comp.graphics.animation
Subject: CALL FOR PRESENTATIONS: Navy SciViz/VR Seminar
Message-ID: <32850@oasys.dt.navy.mil>
Date: 19 Mar 93 20:10:23 GMT
Article-I.D.: oasys.32850
Expires: 30 Apr 93 04:00:00 GMT
Reply-To: lipman@oasys.dt.navy.mil (Robert Lipman)
Followup-To: comp.graphics
Distribution: usa
Organization: Carderock Division, NSWC, Bethesda, MD
Lines: 65


			CALL FOR PRESENTATIONS
	
      NAVY SCIENTIFIC VISUALIZATION AND VIRTUAL REALITY SEMINAR

			Tuesday, June 22, 1993

	    Carderock Division, Naval Surface Warfare Center
	      (formerly the David Taylor Research Center)
			  Bethesda, Maryland

SPONSOR: NESS (Navy Engineering Software System) is sponsori

In [35]:
import spacy 
import en_core_web_sm
nlp = en_core_web_sm.load()

total_tokens=0
for i in (range(15)):
    doc = nlp(pathlib.Path(data_dir / "comp.graphics" / fnames[i]).read_text(encoding="latin-1"))
    doc.__len__()
    total_tokens+=doc.__len__()

average_tokens = total_tokens/15   

print(average_tokens)

297.2


In [8]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['124146', '176845', '176846', '176847', '176849']


In [9]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [10]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

# Mezclando los datos para separarlos en Traning y Test

In [11]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [36]:
print("Ejemplo de train_samples:")
print(train_samples[0])

print("\nEjemplo de val_samples:")
print(val_samples[0])

print("\nEjemplo de train_labels:")
print(train_labels[0])

print("\nEjemplo de val_labels:")
print(val_labels[0])

print("\nNombre de la clase de train_labels[0]:")
print(class_names[train_labels[0]])

print("\nNombre de la clase de val_labels[0]:")
print(class_names[val_labels[0]])


Ejemplo de train_samples:

In article <1993Apr20.032017.5783@wuecl.wustl.edu> jca2@cec1.wustl.edu (Joseph Charles Achkar) writes:
> It was nice to see ESPN show game 1 between the Wings and Leafs since
>the Cubs and Astros got rained out. Instead of showing another baseball
>game, they decided on the Stanley Cup Playoffs. A classy move by ESPN.

They tried their best not to show it, believe me. I'm surprised they
couldn't find a sprint car race (mini cars through pigpens, indeed!)
on short notice.

George
-- 
George Ferguson                 ARPA: ferguson@cs.rochester.edu
Dept. of Computer Science       UUCP: rutgers!rochester!ferguson
University of Rochester         VOX:  (716) 275-2527
Rochester  NY  14627-0226       FAX:  (716) 461-2018


Ejemplo de val_samples:
Lines: 30

In article <1993Apr22.153528.10877@ra.royalroads.ca>,
mlee@post.RoyalRoads.ca (Malcolm Lee) wrote:
> Eternal damnation is the consequence of the choice one makes in rejecting
> God.  If you choose to jump off a cl

In [12]:
print(train_samples[:3])

["\nIn article <1993Apr20.032017.5783@wuecl.wustl.edu> jca2@cec1.wustl.edu (Joseph Charles Achkar) writes:\n> It was nice to see ESPN show game 1 between the Wings and Leafs since\n>the Cubs and Astros got rained out. Instead of showing another baseball\n>game, they decided on the Stanley Cup Playoffs. A classy move by ESPN.\n\nThey tried their best not to show it, believe me. I'm surprised they\ncouldn't find a sprint car race (mini cars through pigpens, indeed!)\non short notice.\n\nGeorge\n-- \nGeorge Ferguson                 ARPA: ferguson@cs.rochester.edu\nDept. of Computer Science       UUCP: rutgers!rochester!ferguson\nUniversity of Rochester         VOX:  (716) 275-2527\nRochester  NY  14627-0226       FAX:  (716) 461-2018\n", "Approved: christian@aramis.rutgers.edu\n\nMy girlfriend is a smoker. She has been addicted to it for quite some time.\nShe has been tried a couple of times, but then always get back to it. Her \nbackground is non-Christian, but she's interested in Christ

In [13]:
print(val_samples[:1])

["Lines: 30\n\nIn article <1993Apr22.153528.10877@ra.royalroads.ca>,\nmlee@post.RoyalRoads.ca (Malcolm Lee) wrote:\n> Eternal damnation is the consequence of the choice one makes in rejecting\n> God.  If you choose to jump off a cliff, you can hardly blame God for you \n> going *splat* at the bottom.  He knows that if you choose to jump, that \n> you will die but He will not prevent you from making that choice.  In fact,\n> He sent His Son to stand on the edge of the cliff and tell everyone of what\n> lies below.  To prove that point, Jesus took that plunge Himself but He being\n> God was able to rise up again.  I have seen the example of Christ and have \n> chosen not to jump and I'm trying to tell you not to jump or else you'll \n> go *splat*.\n>  \n> You don't have to listen to me and I won't stop you if you decide to jump.\n> I only ask that you check it out before taking the plunge.  You owe it to\n> yourself.  I don't like seeing anyone go *splat*.\n\nI'm for the moment intereste

In [14]:
print(train_labels[:1])

[10]


In [15]:
print(val_labels[:1])

[19]


# Tokenización de las palabras con TextVectorization 

In [16]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [17]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [18]:
len(vectorizer.get_vocabulary())

20000

# Viendo la salida de Vectorizer

In [19]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3457, 1682,   15,    2, 5776], dtype=int64)

In [20]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3457, 1682,   15,    2, 5776,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [21]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [22]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3457, 1682, 15, 2, 5776]

# Tokenización de los datos de entrenamiento y validación

In [23]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación y entrenamiento del modelo

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def create_model(vocab_size, sequence_length, num_classes):
    model = keras.Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=sequence_length))
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model


vocab_size = 20000  
sequence_length = 200  
num_classes = len(set(train_labels))  


model = create_model(vocab_size, sequence_length, num_classes)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))




Epoch 1/10


c:\Users\christian.espinosa\Documents\UNIR\TFM\receipt_reader\env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.1273 - loss: 2.8337 - val_accuracy: 0.3828 - val_loss: 2.0247
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.4847 - loss: 1.7605 - val_accuracy: 0.6272 - val_loss: 1.3037
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.6954 - loss: 1.0882 - val_accuracy: 0.7254 - val_loss: 0.9789
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.7813 - loss: 0.7725 - val_accuracy: 0.7697 - val_loss: 0.8075
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.8445 - loss: 0.5704 - val_accuracy: 0.7749 - val_loss: 0.7491
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.8691 - loss: 0.4697 - val_accuracy: 0.7542 - val_loss: 0.7853
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.8818 - loss: 0.4065 - val_accuracy: 0.7664 - val_loss: 0.7306
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.9020 - loss: 0.3322 - val_accuracy: 0.792

# Evaluación

In [25]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(tf.convert_to_tensor([["this message is about computer graphics and 3D modeling"]], dtype=tf.string))
class_names[np.argmax(probabilities[0])]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step


'comp.graphics'

In [26]:
probabilities = end_to_end_model.predict(tf.convert_to_tensor([["politics and federal courts law that people understand with politician and elects congressman"]], dtype=tf.string))
class_names[np.argmax(probabilities[0])]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


'talk.politics.guns'

In [27]:
probabilities = end_to_end_model.predict(tf.convert_to_tensor([["we are talking about religion"]], dtype=tf.string))
class_names[np.argmax(probabilities[0])]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


'comp.sys.mac.hardware'